# NonBDNA Finder — Analysis Notebook

## Overview

This notebook detects and analyses **Non-B DNA structural motifs** in one or more FASTA files using the **NonBDNAFinder** suite.  
Run the single code cell below to perform the complete analysis.

---

## Detectors — 9 classes, 23+ subclasses

| Class | Method | Key Subclasses |
|---|---|---|
| Curved DNA | A-tract phasing | Curved, Bent |
| Slipped DNA | K-mer indexing | Direct-repeat, Mirror |
| Cruciform | Inverted repeat detection | Cruciform |
| R-Loop | QmRLFS algorithm | R-loop, G-rich, C-rich |
| Triplex | Mirror repeat + purine runs | H-DNA, R·R·Y, Y·R·Y |
| G-Quadruplex | G4Hunter + 7 pattern models | G4, Parallel, Anti-parallel… |
| i-Motif | C-run patterns | iM-Canonical, iM-Partial, iM-C-rich |
| Z-DNA | CG/CA repeat scoring | ZH-score, CG-repeat |
| A-philic DNA | 10-mer A-tract patterns | A-philic |
| **Hybrid** | Overlapping-motif detection | Class₁\_Class₂\_Overlap |
| **Non-B Clusters** | Dense multi-motif windows | Mixed\_Cluster\_N\_classes |

---

## What this notebook produces

### Per-file outputs (saved under `OUTPUT_DIR/<timestamp>/<stem>/`)
| File | Contents |
|---|---|
| `motifs.csv` | Every detected motif with Class, Subclass, position, score |
| `motifs.xlsx` | Same data in Excel format |
| `class_distribution.png` | Horizontal bar chart — motif counts per class |
| `subclass_distribution.png` | Horizontal bar chart — motif counts per subclass |
| `hybrid_cluster_breakdown.png` | Bar chart — Hybrid and Cluster motif counts |
| `motif_density_by_sequence.png` | Motifs-per-kb for each sequence (multi-sequence files) |
| `sequence_coverage.png` | Fraction of each sequence covered by Non-B DNA |
| `positional_distribution_<class>.png` | Positional frequency along fixed-length sequences (**equal-length multiFASTA only**) |

### Master outputs (saved under `OUTPUT_DIR/<timestamp>/_master/`)
| File | Contents |
|---|---|
| `master_motifs.csv / .xlsx` | Combined motif table across **all** input files |
| `1_global_class_distribution.csv` | Motif counts by file × class |
| `2_per_file_summary.csv` | Per-file totals (sequences, motifs, classes, hybrids, clusters, coverage, density) |
| `3_class_statistics.csv` | Mean length & score per class |
| `4_file_class_pivot.csv` | Pivot: files × classes |
| `5_subclass_statistics.csv` | Mean length & score per subclass |
| `6_equal_length_positional.csv` | Positional frequency table (equal-length multiFASTA) |
| `master_summary.png` | Global class distribution + file-type breakdown + motifs-per-file |
| `density_comparison.png` | Motif density (motifs per kb) across all input files |
| `coverage_comparison.png` | Sequence coverage (%) across all input files |
| `hybrid_cluster_comparison.png` | Hybrid and Cluster counts across all input files |

### Comparative analysis (saved under `OUTPUT_DIR/<timestamp>/_comparisons/`)
Activated automatically when **≥ 2 files** are present.  
Uses `Species_region.fasta` filename convention when available.

| Plot | Contents |
|---|---|
| `all_comparisons_summary.csv / .xlsx` | Per-file density, GC%, motif lengths |
| Within-species plots | Class dist, subclass dist, density, coverage, length dist, GC% |
| Cross-species heatmaps | Density × region, GC% × region, class comparison per region |

---

## How to use

1. **Edit `FASTA_INPUT`** — a file path, a glob pattern (`*.fasta`), or a Python list of paths/patterns.  
2. **Edit `OUTPUT_DIR`** — destination folder (default: `notebook_reports`).  
3. Optionally restrict detectors with **`ENABLED_CLASSES`** (e.g. `['G-Quadruplex','Z-DNA']`).  
4. **Run the code cell** below — the full analysis runs automatically.

> Results are written to `OUTPUT_DIR/<timestamp>/`.  No internet access is required.


In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# NonBDNAFinder — Complete Analysis
# Edit FASTA_INPUT and OUTPUT_DIR, then run this cell.
# ═══════════════════════════════════════════════════════════════════════════════

# ─────────────────────────────────────────────────────────────────────────────
# 0.  IMPORTS, PACKAGES, CONFIGURATION
# ─────────────────────────────────────────────────────────────────────────────
import sys, os, importlib, glob, gc, time, datetime, re, warnings
import concurrent.futures
from pathlib import Path
from collections import defaultdict
warnings.filterwarnings('ignore')

# Add repo root to Python path
_REPO_ROOT = os.path.abspath(os.getcwd())
if _REPO_ROOT not in sys.path:
    sys.path.insert(0, _REPO_ROOT)

# Auto-install missing packages
_REQUIRED = [
    ('psutil',     'psutil>=5.8.0'),
    ('pandas',     'pandas>=1.3.0'),
    ('numpy',      'numpy>=1.21.0'),
    ('matplotlib', 'matplotlib>=3.5.0'),
    ('seaborn',    'seaborn>=0.11.0'),
    ('openpyxl',   'openpyxl>=3.0.0'),
    ('tqdm',       'tqdm>=4.64.0'),
]
_miss = [p for m, p in _REQUIRED if importlib.util.find_spec(m) is None]
if _miss:
    import subprocess
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', *_miss, '-q'])

import pandas as pd
import numpy as np
# ── Force UTF-8-SIG globally (Excel-safe, prevents Windows charmap errors)
_original_to_csv = pd.DataFrame.to_csv
def _utf8_csv(self, *a, **k):
    k.setdefault("encoding", "utf-8-sig")
    k.setdefault("index", False)
    return _original_to_csv(self, *a, **k)
pd.DataFrame.to_csv = _utf8_csv

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
from tqdm.auto import tqdm
from IPython.display import display, HTML, Image

sns.set_theme(style='whitegrid')

# ── USER CONFIGURATION ────────────────────────────────────────────────────────
FASTA_INPUT        = ['*.fna', '*.fasta']   # path, wildcard glob, or list of paths/globs
OUTPUT_DIR         = 'notebook_reports'
ENABLED_CLASSES    = None                   # None = all 9 detectors; e.g. ['G-Quadruplex','Z-DNA']
RAM_OVERRIDE_BYTES = None                   # None = auto-detect

# ─────────────────────────────────────────────────────────────────────────────
# 1.  GPU DETECTION
# ─────────────────────────────────────────────────────────────────────────────
def _detect_gpu():
    try:
        import torch
        if torch.cuda.is_available():
            return 'cuda', torch.cuda.get_device_name(0)
    except ImportError:
        pass
    try:
        import cupy as cp; cp.array([1])
        return 'cupy', 'CUDA GPU'
    except Exception:
        pass
    return None, None

GPU_BACKEND, GPU_NAME = _detect_gpu()
_gpu_msg = f'GPU  {GPU_BACKEND} ({GPU_NAME})' if GPU_BACKEND else 'GPU  none (CPU only)'
print(f'\u2705 Deps OK | Python {sys.version.split()[0]} | {_gpu_msg}')

# ─────────────────────────────────────────────────────────────────────────────
# 2.  RESOLVE INPUT FILES & CLASSIFY TYPES
# ─────────────────────────────────────────────────────────────────────────────
def _resolve(inp):
    out = []
    for p in ([inp] if isinstance(inp, str) else list(inp)):
        hits = glob.glob(p)
        out.extend(hits)
        if not hits and os.path.isfile(p):
            out.append(p)
    return sorted({str(Path(f).resolve()) for f in out})

FASTA_FILES = _resolve(FASTA_INPUT)
if not FASTA_FILES:
    raise FileNotFoundError(f'No FASTA files found for: {FASTA_INPUT}')

def _seq_lengths(p):
    L, c = [], 0
    with open(p) as fh:
        for ln in fh:
            s = ln.strip()
            if s.startswith('>'):
                if c: L.append(c)
                c = 0
            else:
                c += len(s)
    if c: L.append(c)
    return L

FILE_TYPES = {}   # path -> 'single' | 'multi' | 'multi_equal'
for fp in FASTA_FILES:
    ls = _seq_lengths(fp)
    if   len(ls) == 1:       FILE_TYPES[fp] = 'single'
    elif len(set(ls)) == 1:  FILE_TYPES[fp] = 'multi_equal'
    else:                    FILE_TYPES[fp] = 'multi'

# GFF pairing (same stem, .gff or .gff3)
GFF_MAP = {}
for fp in FASTA_FILES:
    stem, parent = Path(fp).stem, Path(fp).parent
    for ext in ('.gff3', '.gff'):
        candidate = parent / (stem + ext)
        if candidate.exists():
            GFF_MAP[fp] = str(candidate)
            break

print(f'\n\U0001f4c2 Input files: {len(FASTA_FILES)}')
for fp in FASTA_FILES:
    gff_tag = f'  +GFF: {Path(GFF_MAP[fp]).name}' if fp in GFF_MAP else ''
    print(f'   [{FILE_TYPES[fp]:12s}]  {Path(fp).name}{gff_tag}')
print(f'\n\U0001f4c1 Output dir : {OUTPUT_DIR}')

# ─────────────────────────────────────────────────────────────────────────────
# 3.  ADAPTIVE RESOURCE PLANNING
# ─────────────────────────────────────────────────────────────────────────────
from Utilities.system_resource_inspector import SystemResourceInspector
from Utilities.adaptive_chunk_planner    import AdaptiveChunkPlanner
from Utilities.nonbscanner               import analyze_sequence as _nbf_analyze
from Utilities.utilities                 import (
    read_fasta_file,
)

_insp   = SystemResourceInspector()
_budget = RAM_OVERRIDE_BYTES or _insp.get_memory_budget()
_cpus   = _insp.get_cpu_count()
_total  = max(sum(os.path.getsize(f) for f in FASTA_FILES if os.path.exists(f)), 1_000)
_plan   = AdaptiveChunkPlanner().plan(_total, _budget, _cpus)
CHUNK_SIZE, CHUNK_OVERLAP = _plan['chunk_size'], _plan['overlap']
N_WORKERS, EXEC_MODE      = _plan['workers'], _plan['mode']
if GPU_BACKEND:
    N_WORKERS = min(N_WORKERS * 2, os.cpu_count() or 4)

print(f'\u2699\ufe0f  RAM {_budget/1e9:.2f} GB | chunk={CHUNK_SIZE:,} overlap={CHUNK_OVERLAP:,} '
      f'workers={N_WORKERS} mode={EXEC_MODE} gpu={GPU_BACKEND or "none"}')

_RUN_TS = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
_BASE   = Path(OUTPUT_DIR) / _RUN_TS
_BASE.mkdir(parents=True, exist_ok=True)
print(f'\U0001f4c2 Run output: {_BASE}')

# ─────────────────────────────────────────────────────────────────────────────
# 4.  HELPERS
# ─────────────────────────────────────────────────────────────────────────────
def _scan(name, seq):
    return _nbf_analyze(
        sequence=seq, sequence_name=name,
        use_chunking=True,
        chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP,
        use_parallel_chunks=(EXEC_MODE == 'hybrid'),
        enabled_classes=ENABLED_CLASSES,
    )

def _savefig(fig, path, show=True):
    fig.savefig(str(path), dpi=150, bbox_inches='tight')
    plt.close(fig)
    if show:
        display(Image(str(path)))

def _parse_gff(gff_path):
    feats = []
    with open(gff_path) as fh:
        for ln in fh:
            if ln.startswith('#') or not ln.strip():
                continue
            p = ln.rstrip('\n').split('\t')
            if len(p) < 8:
                continue
            try:
                feats.append({
                    'seqid':  p[0], 'type': p[2],
                    'start':  max(int(p[3]) - 1, 0), 'end': int(p[4]),
                    'strand': p[6],
                    'attrs':  p[8] if len(p) > 8 else '',
                })
            except ValueError:
                pass
    return feats

def _gc_and_length(fasta_path):
    gc = total = 0
    with open(fasta_path) as fh:
        for ln in fh:
            s = ln.strip()
            if not s or s.startswith('>'):
                continue
            su = s.upper()
            gc    += su.count('G') + su.count('C')
            total += len(su)
    return (round(gc / total * 100, 2) if total else 0.0), total

def _coverage(df, seq_lengths_dict):
    """Fraction of total sequence bases covered by at least one motif (interval merging)."""
    if df.empty or not seq_lengths_dict:
        return 0.0
    total_len = sum(seq_lengths_dict.values())
    if total_len == 0:
        return 0.0
    covered = 0
    for sname, grp in df.groupby('Sequence_Name'):
        intervals = grp[['Start', 'End']].values.astype(int)
        intervals = intervals[intervals[:, 1] > intervals[:, 0]]
        if len(intervals) == 0:
            continue
        intervals = intervals[np.argsort(intervals[:, 0])]
        s, e = intervals[0]
        for cs, ce in intervals[1:]:
            if cs <= e:
                e = max(e, ce)
            else:
                covered += e - s
                s, e = cs, ce
        covered += e - s
    return round(covered / total_len * 100, 2)

def _safe_fname(s):
    return re.sub(r'[^\w\-]', '_', str(s))

def _parse_species_region(stem):
    idx = stem.find('_')
    if idx == -1:
        return stem, 'unknown'
    return stem[:idx], stem[idx + 1:]

print('\u2705 Engine Ready')


In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# 5.  PER-FILE ANALYSIS
# ─────────────────────────────────────────────────────────────────────────────
RESULTS_BY_FILE = {}   # stem -> {df, folder, file_type, path, seq_lengths}
GFF_RESULTS     = {}   # stem -> {region_df, gff_path, folder}

for fasta_path in tqdm(FASTA_FILES, desc='Files', unit='file'):
    stem     = Path(fasta_path).stem
    ftype    = FILE_TYPES[fasta_path]
    file_dir = _BASE / stem
    file_dir.mkdir(parents=True, exist_ok=True)
    tqdm.write(f'\n\u2500\u2500 {stem}  [{ftype}] \u2500\u2500')

    seqs = read_fasta_file(fasta_path)
    if not seqs:
        tqdm.write('  \u26a0\ufe0f  No sequences \u2014 skipping.')
        continue

    seq_lengths_map = {sn: len(sq) for sn, sq in seqs.items()}

    # ── Whole-genome scan (parallel over sequences) ───────────────────────────
    motifs_file, t0 = [], time.perf_counter()
    with concurrent.futures.ProcessPoolExecutor(max_workers=N_WORKERS) as pool:
        futs = {pool.submit(_scan, sn, sq): sn for sn, sq in seqs.items()}
        for fut in tqdm(concurrent.futures.as_completed(futs),
                        total=len(futs), desc=f'  seqs({stem})', leave=False):
            sn  = futs[fut]
            res = fut.result()
            tqdm.write(f'  \u25b8 {sn[:55]}  \u2192 {len(res):,} motifs')
            motifs_file.extend(res)
    tqdm.write(f'  \u2705 {len(motifs_file):,} motifs in {time.perf_counter()-t0:.1f}s')
    gc.collect()

    # ── Build per-file DataFrame ──────────────────────────────────────────────
    df = pd.DataFrame(motifs_file) if motifs_file else pd.DataFrame()
    for col, dflt in [('Class','Unknown'), ('Subclass','Other'), ('Start',0),
                      ('End',0), ('Length',0), ('Score',0.0), ('Strand','+'),
                      ('Sequence_Name','')]:
        if col not in df.columns:
            df[col] = dflt
    if not df.empty:
        df['Length'] = np.where(
            df['Length'] == 0,
            (df['End'] - df['Start']).clip(lower=0),
            df['Length']
        )
    df['Source_File'] = Path(fasta_path).name
    df['File_Type']   = ftype

    # ── Per-file exports (CSV + Excel only) ───────────────────────────────────
    if not df.empty:
        df.to_csv(str(file_dir / 'motifs.csv'))
        df.to_excel(str(file_dir / 'motifs.xlsx'), index=False)

    # ══════════════════════════════════════════════════════════════════════════
    # PER-FILE PLOTS
    # ══════════════════════════════════════════════════════════════════════════

    if not df.empty:
        # -- 1. Class distribution --------------------------------------------
        cc = df['Class'].value_counts()
        fig, ax = plt.subplots(figsize=(8, max(3, len(cc) * 0.45)))
        ax.barh(cc.index[::-1], cc.values[::-1], color='steelblue')
        ax.set_xlabel('Motif Count')
        ax.set_title(f'{stem} [{ftype}] \u2014 Class Distribution')
        for i, v in enumerate(cc.values[::-1]):
            ax.text(v + 0.5, i, str(v), va='center', fontsize=8)
        plt.tight_layout()
        _savefig(fig, file_dir / 'class_distribution.png', show=False)

        # -- 2. Subclass distribution -----------------------------------------
        sc = df['Subclass'].value_counts().head(30)
        fig, ax = plt.subplots(figsize=(8, max(3, len(sc) * 0.4)))
        ax.barh(sc.index[::-1], sc.values[::-1], color='darkorange')
        ax.set_xlabel('Motif Count')
        ax.set_title(f'{stem} \u2014 Subclass Distribution (top 30)')
        plt.tight_layout()
        _savefig(fig, file_dir / 'subclass_distribution.png', show=False)

        # -- 3. Hybrid & Cluster breakdown ------------------------------------
        _special = df[df['Class'].isin(['Hybrid', 'Non-B_DNA_Clusters'])]
        if not _special.empty:
            sp_cnt = _special['Class'].value_counts()
            fig, ax = plt.subplots(figsize=(6, 3))
            ax.bar(sp_cnt.index, sp_cnt.values, color=['tomato','mediumpurple'])
            ax.set_ylabel('Count')
            ax.set_title(f'{stem} \u2014 Hybrid & Cluster Motifs')
            for i, v in enumerate(sp_cnt.values):
                ax.text(i, v + 0.2, str(v), ha='center', fontsize=9)
            plt.tight_layout()
            _savefig(fig, file_dir / 'hybrid_cluster_breakdown.png', show=False)

        # -- 4. Motif density per sequence (multi-seq files) ------------------
        if ftype in ('multi', 'multi_equal'):
            _dens_rows = []
            for sn, sq_len in seq_lengths_map.items():
                n = len(df[df['Sequence_Name'] == sn])
                _dens_rows.append({'Sequence': sn[:40], 'Length_bp': sq_len,
                                   'Motifs': n,
                                   'Density_per_kb': round(n / sq_len * 1000, 4) if sq_len else 0})
            _dens_df = pd.DataFrame(_dens_rows).sort_values('Density_per_kb', ascending=False)
            _top_dens = _dens_df.head(40)
            fig, ax = plt.subplots(figsize=(9, max(4, len(_top_dens) * 0.35)))
            ax.barh(_top_dens['Sequence'][::-1], _top_dens['Density_per_kb'][::-1],
                    color='teal')
            ax.set_xlabel('Motifs per kb')
            ax.set_title(f'{stem} \u2014 Motif Density by Sequence (top 40)')
            plt.tight_layout()
            _savefig(fig, file_dir / 'motif_density_by_sequence.png', show=False)

        # -- 5. Sequence coverage (%) -----------------------------------------
        if ftype in ('multi', 'multi_equal'):
            _cov_rows = []
            for sn, sq_len in seq_lengths_map.items():
                sub = df[df['Sequence_Name'] == sn]
                if sub.empty or sq_len == 0:
                    _cov_rows.append({'Sequence': sn[:40], 'Coverage_pct': 0.0})
                    continue
                mask = np.zeros(sq_len, dtype=bool)
                for _, row in sub.iterrows():
                    s = max(0, int(row['Start'])); e = min(sq_len, int(row['End']))
                    if e > s: mask[s:e] = True
                _cov_rows.append({'Sequence': sn[:40],
                                  'Coverage_pct': round(mask.sum() / sq_len * 100, 2)})
            _cov_df = pd.DataFrame(_cov_rows).sort_values('Coverage_pct', ascending=False)
            _top_cov = _cov_df.head(40)
            fig, ax = plt.subplots(figsize=(9, max(4, len(_top_cov) * 0.35)))
            ax.barh(_top_cov['Sequence'][::-1], _top_cov['Coverage_pct'][::-1],
                    color='mediumseagreen')
            ax.set_xlabel('Coverage (%)')
            ax.set_title(f'{stem} \u2014 Non-B DNA Coverage by Sequence (top 40)')
            ax.set_xlim(0, 100)
            plt.tight_layout()
            _savefig(fig, file_dir / 'sequence_coverage.png', show=False)

        # -- 6. Positional distribution (equal-length multiFASTA) -------------
        if ftype == 'multi_equal':
            seq_len_val = list(seq_lengths_map.values())[0]
            # Per-class positional frequency
            for cls in df['Class'].unique():
                cls_df = df[df['Class'] == cls]
                starts = cls_df['Start'].dropna().astype(int)
                starts = starts[starts < seq_len_val]
                if starts.empty:
                    continue
                bins = min(100, seq_len_val)
                fig, ax = plt.subplots(figsize=(10, 3))
                ax.hist(starts, bins=bins, color='steelblue', edgecolor='none', alpha=0.8)
                ax.set_xlabel('Position (bp)')
                ax.set_ylabel('Frequency')
                ax.set_title(f'{stem} \u2014 {cls} Positional Distribution '
                             f'(n={len(starts):,}, seq_len={seq_len_val:,})')
                ax.xaxis.set_major_formatter(mticker.FuncFormatter(
                    lambda x, _: f'{int(x):,}'))
                plt.tight_layout()
                _savefig(fig, file_dir / f'positional_dist_{_safe_fname(cls)}.png', show=False)

            # Combined positional distribution (all classes stacked)
            fig, ax = plt.subplots(figsize=(12, 4))
            bins = min(100, seq_len_val)
            for cls in sorted(df['Class'].unique()):
                starts = df[df['Class'] == cls]['Start'].dropna().astype(int)
                starts = starts[starts < seq_len_val]
                if not starts.empty:
                    ax.hist(starts, bins=bins, alpha=0.5, label=cls, histtype='stepfilled')
            ax.set_xlabel('Position (bp)')
            ax.set_ylabel('Frequency')
            ax.set_title(f'{stem} \u2014 Combined Positional Distribution '
                         f'(seq_len={seq_len_val:,})')
            ax.xaxis.set_major_formatter(mticker.FuncFormatter(
                lambda x, _: f'{int(x):,}'))
            ax.legend(fontsize=7, ncol=3)
            plt.tight_layout()
            _savefig(fig, file_dir / 'positional_dist_combined.png', show=False)

        tqdm.write(f'  \U0001f4ca Plots saved to: {file_dir}')

    RESULTS_BY_FILE[stem] = {
        'df': df, 'folder': file_dir, 'file_type': ftype,
        'path': fasta_path, 'seq_lengths': seq_lengths_map,
    }

    # ── GFF region analysis ───────────────────────────────────────────────────
    if fasta_path in GFF_MAP:
        gff_path = GFF_MAP[fasta_path]
        tqdm.write(f'  \U0001f4cb GFF: {Path(gff_path).name}')
        features = _parse_gff(gff_path)
        tqdm.write(f'     {len(features):,} features parsed')

        gff_dir = file_dir / 'gff_regions'
        gff_dir.mkdir(exist_ok=True)

        region_rows = []
        feat_types  = sorted({f['type'] for f in features})
        for ftype_gff in tqdm(feat_types, desc=f'  GFF({stem})', leave=False):
            type_feats  = [f for f in features if f['type'] == ftype_gff]
            type_motifs = []
            for feat in type_feats:
                seq_id     = feat['seqid']
                if seq_id not in seqs:
                    continue
                region_seq = seqs[seq_id][feat['start']:feat['end']]
                if len(region_seq) < 12:
                    continue
                rname = (f"{seq_id}:{ftype_gff}:{feat['start']}-"
                         f"{feat['end']}({feat['strand']})")
                mots = _scan(rname, region_seq)
                for m in mots:
                    m['GFF_Type']   = ftype_gff
                    m['GFF_SeqID']  = seq_id
                    m['GFF_Start']  = feat['start']
                    m['GFF_End']    = feat['end']
                    m['GFF_Strand'] = feat['strand']
                    _a = feat['attrs']
                    m['GFF_Attrs']  = _a[:80] + ('...' if len(_a) > 80 else '')
                type_motifs.extend(mots)
            region_rows.extend(type_motifs)
            tqdm.write(
                f'     {ftype_gff}: {len(type_feats):,} regions \u2192 {len(type_motifs):,} motifs')
            gc.collect()

        gff_df = pd.DataFrame(region_rows) if region_rows else pd.DataFrame()
        for col, dflt in [('Class','Unknown'), ('Subclass','Other'), ('Start',0),
                          ('End',0), ('Length',0), ('Score',0.0),
                          ('GFF_Type',''), ('GFF_SeqID',''),
                          ('GFF_Start',0), ('GFF_End',0),
                          ('GFF_Strand','+'), ('GFF_Attrs','')]:
            if col not in gff_df.columns:
                gff_df[col] = dflt

        if not gff_df.empty:
            gff_df.to_csv(str(gff_dir / 'gff_region_motifs.csv'), index=False)
            pivot = gff_df.groupby(['GFF_Type', 'Class']).size().unstack(fill_value=0)
            fig, ax = plt.subplots(figsize=(max(8, len(pivot) * 1.4), 5))
            pivot.plot(kind='bar', ax=ax, colormap='tab20', width=0.8)
            ax.set_xlabel('GFF Feature Type')
            ax.set_ylabel('Motif Count')
            ax.set_title(f'{stem} \u2014 Motifs per GFF Feature Type')
            ax.legend(title='Class', bbox_to_anchor=(1, 1))
            plt.tight_layout()
            _savefig(fig, gff_dir / 'gff_motifs_by_type.png', show=False)

        GFF_RESULTS[stem] = {'region_df': gff_df, 'gff_path': gff_path, 'folder': gff_dir}
        tqdm.write(f'  \u2705 GFF analysis: {len(gff_df):,} region motifs')

print(f'\n\u2705 Analysis complete \u2014 {len(RESULTS_BY_FILE)} file(s) processed '
      f'({len(GFF_RESULTS)} with GFF).')

# ─────────────────────────────────────────────────────────────────────────────
# 6.  MASTER TABLES & GLOBAL SUMMARY PLOTS
# ─────────────────────────────────────────────────────────────────────────────
_dfs       = [r['df'] for r in RESULTS_BY_FILE.values() if not r['df'].empty]
_master_df = pd.concat(_dfs, ignore_index=True) if _dfs else pd.DataFrame()

_master_dir = _BASE / '_master'
_master_dir.mkdir(exist_ok=True)

_gdfs   = [v['region_df'] for v in GFF_RESULTS.values() if not v['region_df'].empty]
_gff_df = pd.concat(_gdfs, ignore_index=True) if _gdfs else pd.DataFrame()

_tables = {}

if not _master_df.empty:
    _tables['1_global_class_distribution'] = (
        _master_df.groupby(['Source_File', 'File_Type', 'Class'])
        .size().reset_index(name='Count')
    )

    # Per-file summary including hybrid/cluster counts, coverage, density
    _pf_rows = []
    for stem, res in RESULTS_BY_FILE.items():
        df   = res['df']
        fp   = res['path']
        ftype_v = res['file_type']
        sl   = res['seq_lengths']
        gc_pct, seq_len = _gc_and_length(fp)
        n    = len(df)
        density = round(n / seq_len * 1000, 4) if seq_len else 0.0
        cov_pct = _coverage(df, sl)
        n_hybrid   = int((df['Class'] == 'Hybrid').sum())             if not df.empty else 0
        n_clusters = int((df['Class'] == 'Non-B_DNA_Clusters').sum()) if not df.empty else 0
        _pf_rows.append({
            'File':             Path(fp).name,
            'File_Type':        ftype_v,
            'Sequences':        len(sl),
            'Total_bp':         seq_len,
            'GC_Percent':       gc_pct,
            'Total_Motifs':     n,
            'Classes':          df['Class'].nunique()    if not df.empty else 0,
            'Subclasses':       df['Subclass'].nunique() if not df.empty else 0,
            'Hybrids':          n_hybrid,
            'Clusters':         n_clusters,
            'Density_per_kb':   density,
            'Coverage_pct':     cov_pct,
        })
    _tables['2_per_file_summary'] = pd.DataFrame(_pf_rows)

    _tables['3_class_statistics'] = (
        _master_df.groupby('Class')
        .agg(Total_Count=('Class', 'count'),
             Mean_Length=('Length', 'mean'),
             Mean_Score=('Score', 'mean'))
        .round(3).reset_index().sort_values('Total_Count', ascending=False)
    )

    _tables['4_file_class_pivot'] = (
        _master_df.groupby(['Source_File', 'Class'])
        .size().unstack(fill_value=0).reset_index()
    )

    _tables['5_subclass_statistics'] = (
        _master_df.groupby('Subclass')
        .agg(Total_Count=('Subclass', 'count'),
             Mean_Length=('Length', 'mean'),
             Mean_Score=('Score', 'mean'))
        .round(3).reset_index().sort_values('Total_Count', ascending=False)
    )

    # Equal-length multiFASTA positional table
    _eq_dfs = [r['df'] for r in RESULTS_BY_FILE.values()
               if r['file_type'] == 'multi_equal' and not r['df'].empty]
    if _eq_dfs:
        _eq = pd.concat(_eq_dfs, ignore_index=True)
        _tables['6_equal_length_positional'] = (
            _eq.groupby(['Source_File', 'Class', 'Start'])
            .size().reset_index(name='Frequency')
            .sort_values(['Source_File', 'Class', 'Frequency'],
                         ascending=[True, True, False])
        )

if not _gff_df.empty:
    _tables['7_gff_motifs_per_feature_type'] = (
        _gff_df.groupby(['GFF_Type', 'Class'])
        .size().reset_index(name='Count').sort_values('Count', ascending=False)
    )
    _tables['8_gff_density_per_feature'] = (
        _gff_df.assign(Region_Len=(_gff_df['GFF_End'] - _gff_df['GFF_Start']).clip(lower=1))
        .groupby('GFF_Type')
        .agg(Total_Motifs=('Class', 'count'),
             Unique_Classes=('Class', 'nunique'),
             Mean_Region_Len=('Region_Len', 'mean'))
        .round(2).reset_index().sort_values('Total_Motifs', ascending=False)
    )
    _tables['9_gff_class_pivot'] = (
        _gff_df.groupby(['GFF_Type', 'Class']).size().unstack(fill_value=0).reset_index()
    )
    _tables['10_gff_top50_hotspot_regions'] = (
        _gff_df.groupby(['GFF_SeqID', 'GFF_Type', 'GFF_Start', 'GFF_End'])
        .agg(Motif_Count=('Class', 'count'), Classes=('Class', 'nunique'))
        .reset_index().sort_values('Motif_Count', ascending=False).head(50)
    )

# ── Export master tables ──────────────────────────────────────────────────────
if not _master_df.empty:
    _master_df.to_csv(str(_master_dir / 'master_motifs.csv'))
    _master_df.to_excel(str(_master_dir / 'master_motifs.xlsx'), index=False)
if not _gff_df.empty:
    _gff_df.to_csv(str(_master_dir / 'gff_region_motifs_all.csv'), index=False)
for tname, tdf in _tables.items():
    tdf.to_csv(str(_master_dir / f'{tname}.csv'), index=False)

# ── Global summary plots ──────────────────────────────────────────────────────
if not _master_df.empty:
    _pf_summary = _tables.get('2_per_file_summary', pd.DataFrame())
    _n_files    = len(RESULTS_BY_FILE)

    # ---- (a) Global class distribution + file-type pie ----------------------
    _ncol = 2 + (1 if _n_files > 1 else 0) + (1 if not _gff_df.empty else 0)
    fig, axes = plt.subplots(1, _ncol, figsize=(6 * _ncol, 4))
    _ax = iter([axes] if _ncol == 1 else axes.flat)

    cc = _master_df['Class'].value_counts()
    a  = next(_ax)
    a.barh(cc.index[::-1], cc.values[::-1], color='steelblue')
    a.set_xlabel('Count')
    a.set_title('Global Class Distribution')

    ft = _master_df.groupby('File_Type').size()
    a  = next(_ax)
    a.pie(ft.values, labels=ft.index, autopct='%1.1f%%', startangle=90)
    a.set_title('Motifs by File Type')

    if _n_files > 1:
        pf = _master_df.groupby('Source_File').size().sort_values(ascending=False)
        a  = next(_ax)
        a.barh([Path(n).stem[:22] for n in pf.index[::-1]], pf.values[::-1], color='coral')
        a.set_xlabel('Count')
        a.set_title('Motifs per File')

    if not _gff_df.empty:
        a  = next(_ax)
        gt = _gff_df['GFF_Type'].value_counts().head(15)
        a.barh(gt.index[::-1], gt.values[::-1], color='mediumseagreen')
        a.set_xlabel('Count')
        a.set_title('Motifs by GFF Feature Type')

    plt.tight_layout()
    _savefig(fig, _master_dir / 'master_summary.png')

    # ---- (b) Density comparison across files --------------------------------
    if not _pf_summary.empty:
        fig, ax = plt.subplots(figsize=(max(6, len(_pf_summary) * 1.4), 4))
        _pf_s = _pf_summary.sort_values('Density_per_kb', ascending=False)
        bars = ax.bar(_pf_s['File'].apply(lambda x: Path(x).stem[:25]),
                      _pf_s['Density_per_kb'], color='steelblue')
        ax.bar_label(bars, fmt='%.3f', padding=2, fontsize=8)
        ax.set_ylabel('Motifs per kb')
        ax.set_title('Motif Density Comparison Across Files')
        plt.xticks(rotation=30, ha='right')
        plt.tight_layout()
        _savefig(fig, _master_dir / 'density_comparison.png')

    # ---- (c) Coverage comparison across files --------------------------------
    if not _pf_summary.empty:
        fig, ax = plt.subplots(figsize=(max(6, len(_pf_summary) * 1.4), 4))
        _pf_s = _pf_summary.sort_values('Coverage_pct', ascending=False)
        bars = ax.bar(_pf_s['File'].apply(lambda x: Path(x).stem[:25]),
                      _pf_s['Coverage_pct'], color='mediumseagreen')
        ax.bar_label(bars, fmt='%.1f%%', padding=2, fontsize=8)
        ax.set_ylabel('Coverage (%)')
        ax.set_title('Non-B DNA Coverage Comparison Across Files')
        ax.set_ylim(0, 100)
        plt.xticks(rotation=30, ha='right')
        plt.tight_layout()
        _savefig(fig, _master_dir / 'coverage_comparison.png')

    # ---- (d) Hybrid & Cluster comparison ------------------------------------
    if not _pf_summary.empty and (
            _pf_summary['Hybrids'].sum() > 0 or _pf_summary['Clusters'].sum() > 0):
        x      = np.arange(len(_pf_summary))
        width  = 0.35
        labels = _pf_summary['File'].apply(lambda x: Path(x).stem[:20])
        fig, ax = plt.subplots(figsize=(max(7, len(_pf_summary) * 1.5), 4))
        ax.bar(x - width / 2, _pf_summary['Hybrids'],   width, label='Hybrids',  color='tomato')
        ax.bar(x + width / 2, _pf_summary['Clusters'],  width, label='Clusters', color='mediumpurple')
        ax.set_xticks(x); ax.set_xticklabels(labels, rotation=30, ha='right')
        ax.set_ylabel('Count')
        ax.set_title('Hybrid & Cluster Motifs Across Files')
        ax.legend()
        plt.tight_layout()
        _savefig(fig, _master_dir / 'hybrid_cluster_comparison.png')

    # ---- (e) GFF heatmap (feature type x class) -----------------------------
    if not _gff_df.empty and '9_gff_class_pivot' in _tables:
        _piv = _tables['9_gff_class_pivot'].set_index('GFF_Type')
        fig2, ax2 = plt.subplots(figsize=(max(10, len(_piv.columns) * 1.2),
                                          max(4,  len(_piv) * 0.6)))
        sns.heatmap(_piv, annot=True, fmt='d', cmap='YlOrRd', ax=ax2,
                    linewidths=0.4, cbar_kws={'label': 'Motif count'})
        ax2.set_title('GFF Feature Type \u00d7 Non-B Class Heatmap')
        ax2.set_xlabel('Non-B Class')
        ax2.set_ylabel('GFF Feature Type')
        plt.tight_layout()
        _savefig(fig2, _master_dir / 'gff_class_heatmap.png')

    # ---- (f) Subclass distribution (top 30 global) --------------------------
    sc_all = _master_df['Subclass'].value_counts().head(30)
    fig, ax = plt.subplots(figsize=(8, max(4, len(sc_all) * 0.4)))
    ax.barh(sc_all.index[::-1], sc_all.values[::-1], color='darkorange')
    ax.set_xlabel('Count')
    ax.set_title('Global Subclass Distribution (top 30)')
    plt.tight_layout()
    _savefig(fig, _master_dir / 'global_subclass_distribution.png')

# ── Display master tables ─────────────────────────────────────────────────────
for tname, tdf in _tables.items():
    print(f"\n{'='*60}\n{tname.replace('_',' ').upper()}\n{'='*60}")
    display(tdf)

# ─────────────────────────────────────────────────────────────────────────────
# 7.  COMPARATIVE ANALYSIS
# ─────────────────────────────────────────────────────────────────────────────
_sep = '\u2550' * 60

_comp_rows = []
for stem, res in RESULTS_BY_FILE.items():
    species, region = _parse_species_region(stem)
    df  = res['df']
    fp  = res['path']
    sl  = res['seq_lengths']
    gc_pct, seq_len = _gc_and_length(fp)
    n   = len(df)
    density = round(n / seq_len * 1000, 4) if seq_len else 0.0
    cov_pct = _coverage(df, sl)
    _comp_rows.append({
        'Stem':                  stem,
        'Species':               species,
        'Region':                region,
        'Total_Motifs':          n,
        'Seq_Length_bp':         seq_len,
        'Density_per_kb':        density,
        'Coverage_pct':          cov_pct,
        'GC_Percent':            gc_pct,
        'Mean_Motif_Length':     round(df['Length'].mean(),   2) if not df.empty else 0.0,
        'Median_Motif_Length':   round(df['Length'].median(), 2) if not df.empty else 0.0,
        'Unique_Classes':        df['Class'].nunique()    if not df.empty else 0,
        'Unique_Subclasses':     df['Subclass'].nunique() if not df.empty else 0,
        'Hybrids':               int((df['Class'] == 'Hybrid').sum())             if not df.empty else 0,
        'Clusters':              int((df['Class'] == 'Non-B_DNA_Clusters').sum()) if not df.empty else 0,
    })

_comp_df      = pd.DataFrame(_comp_rows)
_species_list = sorted(_comp_df['Species'].unique())
_region_list  = sorted(_comp_df['Region'].unique())
print(f'Species detected : {_species_list}')
print(f'Regions detected : {_region_list}')

_cmp_dir = _BASE / '_comparisons'
_cmp_dir.mkdir(exist_ok=True)

# Save master comparison tables
_comp_df.to_csv(str(_cmp_dir / 'all_comparisons_summary.csv'), index=False)
_comp_df.to_excel(str(_cmp_dir / 'all_comparisons_summary.xlsx'), index=False)

# ── Multi-file comparative plots (even with a single species) ─────────────────
if len(RESULTS_BY_FILE) >= 2:
    # Class comparison across ALL files
    _all_classes = sorted(_master_df['Class'].unique()) if not _master_df.empty else []
    if _all_classes:
        _cls_pivot = _master_df.groupby(['Source_File', 'Class']).size().unstack(fill_value=0)
        fig, ax = plt.subplots(figsize=(max(10, len(_cls_pivot) * 1.4),
                                        max(4,  len(_cls_pivot.columns) * 0.5)))
        _cls_pivot.plot(kind='bar', ax=ax, colormap='tab20', width=0.8)
        ax.set_xlabel('File')
        ax.set_ylabel('Motif Count')
        ax.set_title('Class Distribution \u2014 All Files Comparison')
        ax.legend(title='Class', bbox_to_anchor=(1, 1), fontsize=8)
        plt.xticks(rotation=30, ha='right')
        plt.tight_layout()
        _savefig(fig, _cmp_dir / 'all_files_class_comparison.png')

    # Subclass comparison across ALL files
    _all_subs = sorted(_master_df['Subclass'].unique()) if not _master_df.empty else []
    if _all_subs:
        _sub_pivot = _master_df.groupby(['Source_File', 'Subclass']).size().unstack(fill_value=0)
        # Keep top 20 subclasses by total count for readability
        _top_subs = _master_df['Subclass'].value_counts().head(20).index
        _sub_pivot = _sub_pivot[[c for c in _top_subs if c in _sub_pivot.columns]]
        if not _sub_pivot.empty:
            fig, ax = plt.subplots(figsize=(max(10, len(_sub_pivot) * 1.4),
                                            max(4,  len(_sub_pivot.columns) * 0.4)))
            _sub_pivot.plot(kind='bar', ax=ax, colormap='tab20', width=0.8)
            ax.set_xlabel('File')
            ax.set_ylabel('Motif Count')
            ax.set_title('Subclass Distribution \u2014 All Files (top 20 subclasses)')
            ax.legend(title='Subclass', bbox_to_anchor=(1, 1), fontsize=7)
            plt.xticks(rotation=30, ha='right')
            plt.tight_layout()
            _savefig(fig, _cmp_dir / 'all_files_subclass_comparison.png')

    # Density heatmap (files x classes)
    if not _master_df.empty:
        _fname_to_len = {
            Path(res['path']).name: max(sum(res['seq_lengths'].values()), 1)
            for res in RESULTS_BY_FILE.values()
        }
        _dens_rows_hm = []
        for (fname, cls), grp in _master_df.groupby(['Source_File', 'Class']):
            seq_len = _fname_to_len.get(fname, 1)
            _dens_rows_hm.append({'Source_File': fname, 'Class': cls,
                                  'Density_per_kb': len(grp) / seq_len * 1000})
        _dens_cls = pd.DataFrame(_dens_rows_hm)
        _dens_piv = _dens_cls.pivot_table(
            index='Source_File', columns='Class', values='Density_per_kb', fill_value=0)
        if not _dens_piv.empty:
            fig, ax = plt.subplots(figsize=(max(10, len(_dens_piv.columns) * 1.2),
                                            max(4,  len(_dens_piv) * 0.7)))
            sns.heatmap(_dens_piv, annot=True, fmt='.3f', cmap='YlOrRd', ax=ax,
                        linewidths=0.4, cbar_kws={'label': 'Motifs per kb'})
            ax.set_title('Motif Density Heatmap (motifs per kb) \u2014 Files \u00d7 Classes')
            ax.set_xlabel('Non-B DNA Class')
            ax.set_ylabel('File')
            plt.tight_layout()
            _savefig(fig, _cmp_dir / 'density_heatmap_files_x_classes.png')

# ── Within-species comparison ─────────────────────────────────────────────────
for species in _species_list:
    sp_rows  = _comp_df[_comp_df['Species'] == species].copy()
    sp_stems = sp_rows['Stem'].tolist()
    sp_dir   = _cmp_dir / _safe_fname(species)
    sp_dir.mkdir(exist_ok=True)

    if len(sp_stems) < 2:
        print(f"\n\u26a0  '{species}' has only one region file — skipping within-species plots.")
        continue

    print(f'\n{_sep}\nWithin-species comparison: {species}\n{_sep}')

    # 1. Class distribution by region
    _class_by_region = {}
    for stem in sp_stems:
        df  = RESULTS_BY_FILE[stem]['df']
        reg = sp_rows.loc[sp_rows['Stem'] == stem, 'Region'].values[0]
        _class_by_region[reg] = (
            df['Class'].value_counts() if not df.empty else pd.Series(dtype=int)
        )
    _all_cls = sorted({c for s in _class_by_region.values() for c in s.index})
    if _all_cls:
        _cmat = pd.DataFrame(
            {r: s.reindex(_all_cls, fill_value=0) for r, s in _class_by_region.items()}
        ).T
        fig, ax = plt.subplots(figsize=(max(8, len(_all_cls) * 1.2), 4))
        _cmat.plot(kind='bar', ax=ax, colormap='tab20', width=0.8)
        ax.set_title(f'{species} \u2014 Class Distribution by Region')
        ax.set_xlabel('Region'); ax.set_ylabel('Motif Count')
        ax.legend(title='Class', bbox_to_anchor=(1, 1))
        plt.xticks(rotation=30, ha='right')
        _savefig(fig, sp_dir / 'class_by_region.png', show=False)

    # 2. Subclass distribution by region
    _sub_by_region = {}
    for stem in sp_stems:
        df  = RESULTS_BY_FILE[stem]['df']
        reg = sp_rows.loc[sp_rows['Stem'] == stem, 'Region'].values[0]
        _sub_by_region[reg] = (
            df['Subclass'].value_counts() if not df.empty else pd.Series(dtype=int)
        )
    _all_subs = sorted({c for s in _sub_by_region.values() for c in s.index})
    if _all_subs:
        _smat = pd.DataFrame(
            {r: s.reindex(_all_subs, fill_value=0) for r, s in _sub_by_region.items()}
        ).T
        fig, ax = plt.subplots(figsize=(max(8, len(_all_subs) * 1.0), 4))
        _smat.plot(kind='bar', ax=ax, colormap='tab20', width=0.8)
        ax.set_title(f'{species} \u2014 Subclass Distribution by Region')
        ax.set_xlabel('Region'); ax.set_ylabel('Motif Count')
        ax.legend(title='Subclass', bbox_to_anchor=(1, 1), fontsize=7)
        plt.xticks(rotation=30, ha='right')
        _savefig(fig, sp_dir / 'subclass_by_region.png', show=False)

    # 3. Motif density by region
    fig, ax = plt.subplots(figsize=(max(6, len(sp_rows) * 1.2), 4))
    bars = ax.bar(sp_rows['Region'], sp_rows['Density_per_kb'], color='steelblue')
    ax.bar_label(bars, fmt='%.3f', padding=2)
    ax.set_title(f'{species} \u2014 Motif Density (motifs per kb) by Region')
    ax.set_xlabel('Region'); ax.set_ylabel('Motifs per kb')
    plt.xticks(rotation=30, ha='right')
    _savefig(fig, sp_dir / 'density_by_region.png', show=False)

    # 4. Coverage by region
    fig, ax = plt.subplots(figsize=(max(6, len(sp_rows) * 1.2), 4))
    bars = ax.bar(sp_rows['Region'], sp_rows['Coverage_pct'], color='mediumseagreen')
    ax.bar_label(bars, fmt='%.1f%%', padding=2)
    ax.set_title(f'{species} \u2014 Non-B DNA Coverage (%) by Region')
    ax.set_xlabel('Region'); ax.set_ylabel('Coverage (%)')
    ax.set_ylim(0, 100)
    plt.xticks(rotation=30, ha='right')
    _savefig(fig, sp_dir / 'coverage_by_region.png', show=False)

    # 5. Motif length distribution by region
    _len_parts = []
    for stem in sp_stems:
        df  = RESULTS_BY_FILE[stem]['df']
        reg = sp_rows.loc[sp_rows['Stem'] == stem, 'Region'].values[0]
        if not df.empty and 'Length' in df.columns:
            tmp = df[['Length']].copy()
            tmp['Region'] = reg
            _len_parts.append(tmp)
    if _len_parts:
        _len_df = pd.concat(_len_parts, ignore_index=True)
        _len_df = _len_df[_len_df['Length'] > 0]
        if not _len_df.empty:
            fig, ax = plt.subplots(figsize=(max(8, len(sp_stems) * 2), 4))
            sns.boxplot(data=_len_df, x='Region', y='Length', ax=ax, palette='Set2')
            ax.set_title(f'{species} \u2014 Motif Length Distribution by Region')
            ax.set_xlabel('Region'); ax.set_ylabel('Motif Length (bp)')
            plt.xticks(rotation=30, ha='right')
            _savefig(fig, sp_dir / 'length_by_region.png', show=False)

    # 6. GC content by region
    fig, ax = plt.subplots(figsize=(max(6, len(sp_rows) * 1.2), 4))
    bars = ax.bar(sp_rows['Region'], sp_rows['GC_Percent'], color='goldenrod')
    ax.bar_label(bars, fmt='%.1f%%', padding=2)
    ax.set_title(f'{species} \u2014 GC Content (%) by Region')
    ax.set_xlabel('Region'); ax.set_ylabel('GC %')
    ax.set_ylim(0, 100)
    plt.xticks(rotation=30, ha='right')
    _savefig(fig, sp_dir / 'gc_by_region.png', show=False)

    # 7. Hybrid & Cluster counts by region
    if sp_rows[['Hybrids', 'Clusters']].sum().sum() > 0:
        x     = np.arange(len(sp_rows))
        w     = 0.35
        fig, ax = plt.subplots(figsize=(max(6, len(sp_rows) * 1.4), 4))
        ax.bar(x - w / 2, sp_rows['Hybrids'].values,  w, label='Hybrids',  color='tomato')
        ax.bar(x + w / 2, sp_rows['Clusters'].values, w, label='Clusters', color='mediumpurple')
        ax.set_xticks(x); ax.set_xticklabels(sp_rows['Region'], rotation=30, ha='right')
        ax.set_ylabel('Count')
        ax.set_title(f'{species} \u2014 Hybrid & Cluster Motifs by Region')
        ax.legend()
        plt.tight_layout()
        _savefig(fig, sp_dir / 'hybrid_cluster_by_region.png', show=False)

    # Summary table
    _sp_summary = sp_rows.set_index('Region')[[
        'Total_Motifs', 'Seq_Length_bp', 'Density_per_kb', 'Coverage_pct',
        'GC_Percent', 'Mean_Motif_Length', 'Median_Motif_Length',
        'Unique_Classes', 'Unique_Subclasses', 'Hybrids', 'Clusters',
    ]]
    _sp_summary.to_csv(str(sp_dir / 'within_species_summary.csv'))
    print(f'\n{species} \u2014 Summary')
    display(_sp_summary)

# ── Cross-species comparison ──────────────────────────────────────────────────
if len(_species_list) >= 2:
    _xs_dir = _cmp_dir / '_cross_species'
    _xs_dir.mkdir(exist_ok=True)

    print(f'\n{_sep}\nCross-species comparison: {_species_list}\n{_sep}')

    _sp_region_sets = {
        sp: set(_comp_df[_comp_df['Species'] == sp]['Region'])
        for sp in _species_list
    }
    _shared_regions = sorted(set.intersection(*_sp_region_sets.values()))
    _all_regions    = sorted(set.union(*_sp_region_sets.values()))
    print(f'Shared regions : {_shared_regions}')
    print(f'All regions    : {_all_regions}')

    # 1. Density heatmap (species x region)
    _dens_pivot = _comp_df.pivot_table(
        index='Species', columns='Region',
        values='Density_per_kb', aggfunc='mean')
    if not _dens_pivot.empty:
        fig, ax = plt.subplots(figsize=(max(8, len(_all_regions) * 1.4),
                                        max(4, len(_species_list) * 0.8)))
        sns.heatmap(_dens_pivot, annot=True, fmt='.3f', cmap='YlOrRd', ax=ax,
                    linewidths=0.4, cbar_kws={'label': 'Motifs per kb'})
        ax.set_title('Cross-Species \u2014 Motif Density Heatmap (motifs per kb)')
        ax.set_xlabel('Region'); ax.set_ylabel('Species')
        _savefig(fig, _xs_dir / 'cross_species_density_heatmap.png', show=False)

    # 2. Coverage heatmap (species x region)
    _cov_pivot = _comp_df.pivot_table(
        index='Species', columns='Region',
        values='Coverage_pct', aggfunc='mean')
    if not _cov_pivot.empty:
        fig, ax = plt.subplots(figsize=(max(8, len(_all_regions) * 1.4),
                                        max(4, len(_species_list) * 0.8)))
        sns.heatmap(_cov_pivot, annot=True, fmt='.1f', cmap='Blues', ax=ax,
                    linewidths=0.4, cbar_kws={'label': 'Coverage %'})
        ax.set_title('Cross-Species \u2014 Non-B DNA Coverage Heatmap (%)')
        ax.set_xlabel('Region'); ax.set_ylabel('Species')
        _savefig(fig, _xs_dir / 'cross_species_coverage_heatmap.png', show=False)

    # 3. GC% heatmap (species x region)
    _gc_pivot = _comp_df.pivot_table(
        index='Species', columns='Region',
        values='GC_Percent', aggfunc='mean')
    if not _gc_pivot.empty:
        fig, ax = plt.subplots(figsize=(max(8, len(_all_regions) * 1.4),
                                        max(4, len(_species_list) * 0.8)))
        sns.heatmap(_gc_pivot, annot=True, fmt='.1f', cmap='YlGn', ax=ax,
                    linewidths=0.4, cbar_kws={'label': 'GC %'})
        ax.set_title('Cross-Species \u2014 GC Content Heatmap (%)')
        ax.set_xlabel('Region'); ax.set_ylabel('Species')
        _savefig(fig, _xs_dir / 'cross_species_gc_heatmap.png', show=False)

    # 4. Class comparison per shared region
    for region in _shared_regions:
        _rc = {}
        for stem in _comp_df[_comp_df['Region'] == region]['Stem']:
            df = RESULTS_BY_FILE[stem]['df']
            sp = _comp_df.loc[_comp_df['Stem'] == stem, 'Species'].values[0]
            _rc[sp] = df['Class'].value_counts() if not df.empty else pd.Series(dtype=int)
        _all_cls = sorted({c for s in _rc.values() for c in s.index})
        if _all_cls:
            _rmat = pd.DataFrame(
                {sp: s.reindex(_all_cls, fill_value=0) for sp, s in _rc.items()}
            ).T
            fig, ax = plt.subplots(figsize=(max(8, len(_all_cls) * 1.2), 4))
            _rmat.plot(kind='bar', ax=ax, colormap='tab20', width=0.8)
            ax.set_title(f'Cross-Species Class Comparison \u2014 {region}')
            ax.set_xlabel('Species'); ax.set_ylabel('Motif Count')
            ax.legend(title='Class', bbox_to_anchor=(1, 1))
            plt.xticks(rotation=30, ha='right')
            _savefig(fig, _xs_dir / f'cross_species_class_{_safe_fname(region)}.png',
                     show=False)

    # 5. Length distribution for shared regions
    _xs_len_parts = []
    for stem, res in RESULTS_BY_FILE.items():
        row = _comp_df[_comp_df['Stem'] == stem]
        if row.empty:
            continue
        sp     = row['Species'].values[0]
        region = row['Region'].values[0]
        if region not in _shared_regions:
            continue
        df = res['df']
        if not df.empty and 'Length' in df.columns:
            tmp = df[['Length']].copy()
            tmp['Species'] = sp
            tmp['Region']  = region
            _xs_len_parts.append(tmp)
    if _xs_len_parts:
        _xs_len_df = pd.concat(_xs_len_parts, ignore_index=True)
        _xs_len_df = _xs_len_df[_xs_len_df['Length'] > 0]
        if not _xs_len_df.empty:
            _w = max(10, len(_shared_regions) * len(_species_list) * 1.5)
            fig, ax = plt.subplots(figsize=(_w, 5))
            sns.boxplot(data=_xs_len_df, x='Region', y='Length',
                        hue='Species', ax=ax, palette='Set2')
            ax.set_title('Cross-Species \u2014 Motif Length Distribution by Region')
            ax.set_xlabel('Region'); ax.set_ylabel('Motif Length (bp)')
            plt.xticks(rotation=30, ha='right')
            ax.legend(title='Species', bbox_to_anchor=(1, 1))
            _savefig(fig, _xs_dir / 'cross_species_length_by_region.png', show=False)

    # 6. Hybrid & Cluster cross-species
    _xs_summary = _comp_df[[
        'Species', 'Region', 'Total_Motifs', 'Seq_Length_bp',
        'Density_per_kb', 'Coverage_pct', 'GC_Percent',
        'Mean_Motif_Length', 'Median_Motif_Length',
        'Unique_Classes', 'Unique_Subclasses', 'Hybrids', 'Clusters',
    ]].sort_values(['Species', 'Region'])
    _xs_summary.to_csv(str(_xs_dir / 'cross_species_summary.csv'), index=False)
    print('\nCross-Species Summary Table')
    display(_xs_summary)

# ─────────────────────────────────────────────────────────────────────────────
# 8.  DOWNLOAD LINKS
# ─────────────────────────────────────────────────────────────────────────────
import base64

_MIME = {
    'csv':  'text/csv',
    'xlsx': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',
    'png':  'image/png',
}

def _dl(path, label):
    with open(path, 'rb') as fh:
        b64 = base64.b64encode(fh.read()).decode()
    ext  = Path(path).suffix.lstrip('.')
    mime = _MIME.get(ext, 'application/octet-stream')
    return (f'<a href="data:{mime};base64,{b64}" download="{Path(path).name}" '
            f'style="margin:2px 6px;padding:3px 8px;border:1px solid #aaa;'
            f'border-radius:4px;text-decoration:none;">{label}</a>')

_html = ['<h2>\U0001f4e5 Downloads</h2><h3>Master Outputs</h3><div>']
for fmt, fn in [('CSV', 'master_motifs.csv'), ('Excel', 'master_motifs.xlsx')]:
    p = _master_dir / fn
    if p.exists(): _html.append(_dl(str(p), f'Master {fmt}'))
if (_master_dir / 'gff_region_motifs_all.csv').exists():
    _html.append(_dl(str(_master_dir / 'gff_region_motifs_all.csv'), 'GFF Regions CSV'))
_html.append('</div><h3>Summary Tables</h3><div>')
for tn in _tables:
    p = _master_dir / f'{tn}.csv'
    if p.exists(): _html.append(_dl(str(p), tn.replace('_', ' ').title()))
_html.append('</div><h3>Comparative Analysis</h3><div>')
for fn in ['all_comparisons_summary.csv', 'all_comparisons_summary.xlsx']:
    p = _cmp_dir / fn
    if p.exists(): _html.append(_dl(str(p), fn))
_html.append('</div><h3>Per-File Outputs</h3>')
for stem, res in RESULTS_BY_FILE.items():
    _html.append(f'<details style="margin:4px 0"><summary><b>{stem}</b>'
                 f' <em>[{res["file_type"]}]</em></summary>'
                 f'<div style="margin:4px 12px">')
    for fmt, fn in [('CSV', 'motifs.csv'), ('Excel', 'motifs.xlsx')]:
        p = res['folder'] / fn
        if p.exists(): _html.append(_dl(str(p), fmt))
    for fn in sorted((res['folder']).glob('*.png')):
        _html.append(_dl(str(fn), fn.stem.replace('_', ' ').title()))
    _html.append('</div></details>')

display(HTML('\n'.join(_html)))
print(f'\n\u2705 All outputs saved to: {_BASE}')
